<a href="https://colab.research.google.com/github/taaha3244/HuggingFace-NLP/blob/main/Prompt_Engineering_In_context_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
!pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
huggingface_dataset_name="knkarthick/dialogsum"
dataset=load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
example_indices=[40]
dash_line="-".join('' for x in range(100) )

for i,index in enumerate(example_indices):
  print(dash_line)
  print("Example",i + 1)
  print(dash_line)
  print('INPUT DIALOGUE:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('BASE LINE HUMAN SUMMARY:')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()




---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASE LINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------



In [5]:
model_name='google/flan-t5-small'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [6]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [7]:
sentence='My name is Taaha'
sentence_encoded=tokenizer(sentence,return_tensors='pt')
sentence_decoded=tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True

)

print('Encoded Sentence : ')
print(sentence_encoded['input_ids'][0])
print('\nDecoded sentence :')
print(sentence_decoded)

Encoded Sentence : 
tensor([ 499,  564,   19, 2067,    9, 1024,    1])

Decoded sentence :
My name is Taaha


In [9]:
for i,index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  inputs=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example',i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'Model Generation-Without prompt engineering:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation-Without prompt engineering:
#Person1#: I'm sorry, but I'm not sure if it's a good time to 

In [10]:
for i,index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Summarize the following conversation

{dialogue}

Summary:
  """

  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example',i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'Model Generation-Zero Shot:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation-Zero Shot:
How long is it?



In [11]:
for i,index in enumerate(example_indices):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Dialogue:

{dialogue}

Explain what is going on in the above dialogues in detail
  """

  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print('Example',i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'Model Generation-Zero Shot:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation-Zero Shot:
Tom is going to the railway station. He must catch the train.



In [41]:
def make_prompt(example_indices_full,example_index_to_summarize):
  prompt=''
  for i,index in enumerate(example_indices_full):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    prompt+=f"""
Dialogue:

{dialogue}

Explain what is going on in the above dialogues in detail
{summary}
"""
  dialogue=dataset['test'][example_index_to_summarize]['dialogue']
  prompt+=f"""
Dialogue:

{dialogue}

Explain what is going on in the above dialogues in detail
"""
  return prompt

In [44]:
example_indices_full=[40]
example_index_to_summarize=200

example=make_prompt(example_indices_full,example_index_to_summarize)
print(example)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Explain what is going on in the above dialogues in detail
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster proc

In [45]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(example,return_tensors='pt')
output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'Model Generation-One Shot:\n{output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model Generation-One Shot:
#Person1#: I'm not sure what to expect from your software. I'd like to make a flyer and banner for advertising. I'd like to add a CD-ROM drive.



In [46]:
example_indices_full1=[40,80,120]
example_index_to_summarize=200

example=make_prompt(example_indices_full1,example_index_to_summarize)
print(example)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Explain what is going on in the above dialogues in detail
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and

In [48]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(example,return_tensors='pt')
output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=50
      )[0],
      skip_special_tokens=True
  )

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'Model Generation-Few Shot:\n{output}\n')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
Model Generation-Few Shot:
#Person1#: I'm not sure what I would need. I'd like to upgrade my computer. I'd like to add a CD-ROM drive.

